# Text Analytics Pipeline for Text Classification

This notebook demonstrates how to build a text analytics pipeline that includes text processing, feature extraction, classification, and evaluation.


In [ ]:
# import pandas as pd

# # Read the dataset
# df = pd.read_csv("../Data/labelled_data_1.csv")

# # Define the classes in the desired order (adjust if needed)
# classes = ['neutral', 'negative', 'positive']

# # Total number of records desired
# total_records = 500
# n_classes = len(classes)

# # Calculate base count per class and remainder
# base_count = total_records // n_classes      # e.g. 500 // 3 = 166
# remainder = total_records % n_classes          # e.g. 500 % 3 = 2

# selected_dfs = []
# for i, cls in enumerate(classes):
#     # Filter records for the current class
#     cls_df = df[df['label_1'] == cls].copy()
    
#     # Sort by the similarity field in descending order (highest similarity first)
#     cls_df = cls_df.sort_values(by='similarity', ascending=False)
    
#     # Determine number of records to pick for this class.
#     # For the first "remainder" classes, add one extra record.
#     n_records = base_count + (1 if i < remainder else 0)
    
#     # Take the top n_records from the sorted dataframe
#     selected_dfs.append(cls_df.head(n_records))

# # Combine the subsets for each class into one dataframe
# balanced_subset = pd.concat(selected_dfs)

# # Optionally, shuffle the combined dataframe
# balanced_subset = balanced_subset.sample(frac=1, random_state=42).reset_index(drop=True)

# print("Balanced subset shape:", balanced_subset.shape)
# print(balanced_subset.head())
# balanced_subset.to_csv("../Data/TestDataOnly.csv", index=False)


In [ ]:
# %pip install pandas numpy nltk emoji scikit-learn
# !python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer  # corrected typo: WorldNetLemmatizer -> WordNetLemmatizer
from nltk.corpus import stopwords
import emoji
import spacy
import contractions

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Custom Text Preprocessor

The custom transformer below:

 - **Emoji Conversion:** Converts any emojis to their text descriptions.
 - **Normalization:** Lowercases the text.
 - **Punctuation Removal:** Removes punctuation using regex.
 - **Tokenization:** Uses NLTK’s `word_tokenize`.
 - **Stop-word Removal:** Filters out English stopwords.
 - **Stemming:** Applies Porter stemming.
 
 The transformer implements `fit` and `transform` so that it can be used inside a scikit-learn pipeline.

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, do_stemming=True, do_lemmatization=False, remove_stopwords=True, 
                 do_emoji_conversion=True, use_spacy_tokenizer=True):
        """
        Parameters:
        - do_stemming: Apply stemming (reduces words to their root form)
        - do_lemmatization: Apply lemmatization (converts words to their canonical form)
          Note: When using the default (NLTK) tokenizer, if both do_lemmatization and do_stemming are enabled,
          lemmatization takes precedence.
        - remove_stopwords: Remove common stopwords
        - do_emoji_conversion: Convert emojis to text descriptions
        - use_spacy_tokenizer: Use a custom spaCy-based tokenizer (which already uses lemmatization)
        """
        self.do_stemming = do_stemming
        self.do_lemmatization = do_lemmatization
        self.remove_stopwords = remove_stopwords
        self.do_emoji_conversion = do_emoji_conversion
        self.use_spacy_tokenizer = use_spacy_tokenizer
        self.stemmer = PorterStemmer()
        if self.do_lemmatization:
            self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Load the spaCy model if using the spaCy tokenizer
        if self.use_spacy_tokenizer:
            self.nlp = spacy.load("en_core_web_sm")
    
    def remove_links(self, text):
        """Remove URLs from text."""
        return re.sub(r'http[s]?://\S+|www\.\S+', '', text)
    
    def remove_user_mentions(self, text):
        """Remove user mentions from text."""
        return re.sub(r'u/\S+', '', text)
    
    def expand_contractions(self, text):
        """Expand contractions in the text."""
        return contractions.fix(text)
    
    def remove_non_ascii(self, text):
        """Remove non-ASCII characters from the text."""
        return text.encode("ascii", "ignore").decode()
    
    def remove_punctuations(self, text):
        """
        Remove or adjust punctuation in text.
        Replaces hyphens with space and ensures separation around punctuation.
        """
        text = re.sub(r'[-]', ' ', text)
        text = re.sub(r'(\S)[' + re.escape(string.punctuation) + r'](\S)', r'\1 \2', text)
        return text
    
    def remove_numbers(self, text):
        """Remove numbers from text."""
        return re.sub(r'[0-9]+', '', text)
    
    def emoji_to_text(self, text):
        """Convert emojis to text descriptions."""
        return emoji.demojize(text)
    
    def normalize(self, text):
        """Lowercase the text."""
        return text.lower()
    
    def tokenize(self, text):
        """
        Tokenize text using either a spaCy-based custom tokenizer or the default NLTK tokenizer.
        """
        if self.use_spacy_tokenizer:
            # Use spaCy's custom tokenization logic:
            doc = self.nlp(text)
            tokens = []
            # Add named entities as tokens
            for ent in doc.ents:
                tokens.append(ent.text)
            # Add non-entity tokens using their lemma
            non_entity_tokens = [token.lemma_.lower() for token in doc if not token.ent_type_ 
                                 and not token.is_punct and not token.is_space]
            tokens.extend(non_entity_tokens)
            if self.remove_stopwords:
                tokens = [token for token in tokens if token.lower() not in self.stop_words]
            if self.do_stemming:
                tokens = [self.stemmer.stem(token) for token in tokens]
            return tokens
        else:
            # Default NLTK-based tokenization:
            # Remove punctuation (if any remains) and then tokenize
            text = re.sub(r'[^\w\s]', '', text)
            tokens = word_tokenize(text)
            # Keep only alphabetic tokens
            tokens = [token for token in tokens if token.isalpha()]
            if self.remove_stopwords:
                tokens = [token for token in tokens if token.lower() not in self.stop_words]
            # Apply lemmatization if enabled; otherwise, apply stemming if enabled
            if self.do_lemmatization:
                tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
            elif self.do_stemming:
                tokens = [self.stemmer.stem(token) for token in tokens]
            return tokens
    
    def preprocess(self, text):
        """Apply the complete preprocessing pipeline to the text."""
        text = self.remove_links(text)
        text = self.remove_user_mentions(text)
        text = self.expand_contractions(text)
        text = self.remove_non_ascii(text)
        text = self.remove_punctuations(text)
        text = self.remove_numbers(text)
        if self.do_emoji_conversion:
            text = self.emoji_to_text(text)
        text = self.normalize(text)
        tokens = self.tokenize(text)
        return ' '.join(tokens)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.apply(self.preprocess)

 ## Data Loading and Train/Test Split
 
 We load the dataset and split it into training and testing sets.

In [ ]:
# Read the dataset (make sure the file is in your working directory)
df = pd.read_csv("../Data/TestDataOnly.csv")

# Check available columns
print("Columns in dataset:", df.columns.tolist())

# Select the important columns and drop any missing values
df = df[['text', 'label_1']].dropna()
X = df['text']
y = df['label_1']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Building Various Pipelines
 
 We create several pipelines:
 
 1. **CountVectorizer with Unigrams (Binary Representation):**  
    Uses binary occurrence of words.

 2. **CountVectorizer with Unigrams (Binary Representation):**  
    Uses binary occurrence of words.
 
 3. **TfidfVectorizer with Unigrams:**  
    Uses TF-IDF weights for unigrams.
 
 3. **TfidfVectorizer with N-grams:**  
    Uses TF-IDF weights for unigrams and bigrams.
 
 For each representation, we create 4 classifiers: Logistic Regression, SVM (using LinearSVC), RandomForest and Multtinomial Naive Bayes

In [ ]:
#  Model Pipelines (Binary and TF-IDF, with and without SVD)


# --- Logistic Regression Pipelines (with weighted balancing) ---
pipeline_lr_count_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,1))),
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

pipeline_lr_count_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

pipeline_lr_tfidf_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

pipeline_lr_tfidf_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])


# --- SVM Pipelines (using LinearSVC with weighted balancing) ---
pipeline_svm_count_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,1))),
    ('classifier', LinearSVC(max_iter=1000, class_weight='balanced'))
])

pipeline_svm_count_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('classifier', LinearSVC(max_iter=1000, class_weight='balanced'))
])

pipeline_svm_tfidf_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', LinearSVC(max_iter=1000, class_weight='balanced'))
])

pipeline_svm_tfidf_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', LinearSVC(max_iter=1000, class_weight='balanced'))
])


# --- Random Forest Pipelines (with weighted balancing) ---
pipeline_rf_count_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,1))),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

pipeline_rf_count_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

pipeline_rf_tfidf_unigram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

pipeline_rf_tfidf_ngram = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])


# --- Naive Bayes Pipelines (with Random Under-Sampling) ---
pipeline_nb_count_unigram = ImbPipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,1))),
    ('sampler', RandomUnderSampler(random_state=42)),
    ('classifier', MultinomialNB())
])

pipeline_nb_count_ngram = ImbPipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', CountVectorizer(binary=True, ngram_range=(1,2))),
    ('sampler', RandomUnderSampler(random_state=42)),
    ('classifier', MultinomialNB())
])

pipeline_nb_tfidf_unigram = ImbPipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,1))),
    ('sampler', RandomUnderSampler(random_state=42)),
    ('classifier', MultinomialNB())
])

pipeline_nb_tfidf_ngram = ImbPipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('sampler', RandomUnderSampler(random_state=42)),
    ('classifier', MultinomialNB())
])


### In this section we will define and add SVD to the pipelines (TruncatedSVD) for dimensionality reduction


In [ ]:
# Define Helper to Insert SVD
# ---------------------------
def add_svd(pipeline, n_components=100):
    """
    Inserts a TruncatedSVD step right after the vectorizer.
    Assumes the pipeline has steps: preprocessor, vectorizer, classifier.
    """
    steps = pipeline.steps.copy()
    # Insert SVD at position 2 (right after vectorizer)
    steps.insert(2, ('svd', TruncatedSVD(n_components=n_components)))
    return Pipeline(steps)


# Create pipelines without SVD for NB
pipelines_no_svd = {
    # For models that don't use SVD (or for NB)
    "NB_Count_Binary_Unigram": pipeline_nb_count_unigram,
    "NB_Count_Binary_Ngram": pipeline_nb_count_ngram,
    "NB_Tfidf_Unigram": pipeline_nb_tfidf_unigram,
    "NB_Tfidf_Ngram": pipeline_nb_tfidf_ngram,
}

# For models other than MultinomialNB, with their SVD versions
other_pipelines = {
    "LR_Count_Binary_Unigram": pipeline_lr_count_unigram,
    "LR_Count_Binary_Ngram": pipeline_lr_count_ngram,
    "LR_Tfidf_Unigram": pipeline_lr_tfidf_unigram,
    "LR_Tfidf_Ngram": pipeline_lr_tfidf_ngram,
    
    "SVM_Count_Binary_Unigram": pipeline_svm_count_unigram,
    "SVM_Count_Binary_Ngram": pipeline_svm_count_ngram,
    "SVM_Tfidf_Unigram": pipeline_svm_tfidf_unigram,
    "SVM_Tfidf_Ngram": pipeline_svm_tfidf_ngram,
    
    "RF_Count_Binary_Unigram": pipeline_rf_count_unigram,
    "RF_Count_Binary_Ngram": pipeline_rf_count_ngram,
    "RF_Tfidf_Unigram": pipeline_rf_tfidf_unigram,
    "RF_Tfidf_Ngram": pipeline_rf_tfidf_ngram,
}

# Create SVD versions for non-NB pipelines
svd_pipelines = {name + "_SVD": add_svd(pipe) for name, pipe in other_pipelines.items()}

# Combine all pipelines
all_pipelines = {}
all_pipelines.update(pipelines_no_svd)
all_pipelines.update(other_pipelines)
all_pipelines.update(svd_pipelines)

## Evaluating the Pipelines
 
 We define a helper function that fits a pipeline and returns evaluation metrics:
 
 - **Accuracy**  
 - **Precision** (weighted)  
 - **Recall** (weighted)  
 - **F1 Score** (weighted)
 
 Then, we loop over all pipelines, evaluate them on the test set, and compile the results into a comparison table.

In [ ]:
def evaluate_pipeline_metrics(pipeline, X_train, X_test, y_train, y_test):
    """Train the pipeline and return evaluation metrics."""
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    metrics = {
        "Accuracy": accuracy_score(y_test, predictions),
        "Precision": precision_score(y_test, predictions, average='weighted', zero_division=0),
        "Recall": recall_score(y_test, predictions, average='weighted', zero_division=0),
        "F1 Score": f1_score(y_test, predictions, average='weighted', zero_division=0)
    }
    return metrics


# Evaluate each pipeline and store results
results = []
for name, pipe in all_pipelines.items():
    metrics = evaluate_pipeline_metrics(pipe, X_train, X_test, y_train, y_test)
    row = {"Pipeline": name}
    row.update(metrics)
    results.append(row)

# Create a DataFrame of results and sort by F1 Score
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="F1 Score", ascending=False)
print("### Model Comparison Table")
results_df.reset_index(drop=True, inplace=True)
print(results_df)

## Cross-Validation Evaluation on Training Set

In [ ]:
print("\n### 5-Fold Cross-Validation (F1 Macro) Scores") # Just kept F1 - Macro, can be changed to other metrics
for name, pipe in all_pipelines.items():
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1_macro', n_jobs=1)
    print(f"{name:35s}: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

## Parameter Tuning with GridSearchCV [TO BE VERIFIED]
 
 Here, we perform grid search on a pipeline using `TfidfVectorizer` to tune parameters such as:
 
 - **ngram_range:** Unigrams vs. unigrams+bigrams.
 - **use_idf:** Whether to use the inverse document frequency reweighting.
 - **C:** Regularization strength for Logistic Regression.
 
 The grid search uses 5-fold cross-validation and optimizes for macro F1 score.


In [ ]:
pipeline_grid = ImbPipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer()),  # We'll tune various vectorizer parameters here
    ('svd', 'passthrough'),             # Optional dimensionality reduction step
    ('sampler', 'passthrough'),         # Placeholder; will be set to RandomUnderSampler for NB in grid search
    ('classifier', LogisticRegression(max_iter=1000))
])

# Define a parameter grid that explores various options including different classifiers and balancing.
param_grid = [
    {
        # Parameters for LogisticRegression (using class_weight balancing)
        'vectorizer__ngram_range': [(1,1), (1,2)],
        'vectorizer__use_idf': [True, False],
        'vectorizer__max_df': [0.7, 0.8, 0.9, 1.0],
        'vectorizer__min_df': [1, 2, 3, 5],
        'vectorizer__norm': ['l1', 'l2', None],
        'vectorizer__sublinear_tf': [True, False],
        'svd': [TruncatedSVD(n_components=50), 
                TruncatedSVD(n_components=100), 
                TruncatedSVD(n_components=200), 
                'passthrough'],
        'sampler': ['passthrough'],  # No sampling needed when using class_weight
        'classifier': [LogisticRegression(max_iter=1000, solver='saga')],
        'classifier__C': [0.01, 0.1, 1, 10],
        'classifier__class_weight': ['balanced']
    },
    {
        # Parameters for LinearSVC (using class_weight balancing)
        'vectorizer__ngram_range': [(1,1), (1,2)],
        'vectorizer__use_idf': [True, False],
        'vectorizer__max_df': [0.7, 0.8, 0.9, 1.0],
        'vectorizer__min_df': [1, 2, 3, 5],
        'vectorizer__norm': ['l1', 'l2', None],
        'svd': [TruncatedSVD(n_components=50), 
                TruncatedSVD(n_components=100), 
                TruncatedSVD(n_components=200), 
                'passthrough'],
        'sampler': ['passthrough'],
        'classifier': [LinearSVC(max_iter=1000)],
        'classifier__C': [0.01, 0.1, 1, 10],
        'classifier__loss': ['hinge', 'squared_hinge'],
        'classifier__class_weight': ['balanced']
    },
    {
        # Parameters for RandomForestClassifier (using class_weight balancing)
        'vectorizer__ngram_range': [(1,1), (1,2)],
        'vectorizer__use_idf': [True, False],
        'svd': [TruncatedSVD(n_components=50), 
                TruncatedSVD(n_components=100), 
                TruncatedSVD(n_components=200), 
                'passthrough'],
        'sampler': ['passthrough'],
        'classifier': [RandomForestClassifier(random_state=42)],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [None, 5, 10, 20],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__class_weight': ['balanced']
    },
    {
        # Parameters for MultinomialNB (avoiding SVD to prevent negative values)
        # Use RandomUnderSampler for balancing
        'vectorizer__ngram_range': [(1,1), (1,2)],
        'vectorizer__use_idf': [True, False],
        'vectorizer__max_df': [0.9, 1.0],
        'vectorizer__min_df': [1, 2],
        'svd': ['passthrough'],  # No SVD for NB
        'sampler': [RandomUnderSampler(random_state=42)],  # Apply under-sampling to balance classes
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
        'classifier__fit_prior': [True, False]
    }
]

# Create the GridSearchCV object using 5-fold cross-validation on the training set
grid_search = GridSearchCV(pipeline_grid, param_grid, cv=5, scoring='f1_macro', n_jobs=1, verbose=1)
grid_search.fit(X_train, y_train)

print("\nBest Parameters from Grid Search:")
print(grid_search.best_params_)

### Evaluating the best parameters on 20% test set (from 80 - 20 split)

In [ ]:
# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions_best = best_model.predict(X_test)
print("\n### Evaluation of Best Model from Grid Search")
print(classification_report(y_test, predictions_best))